# Causal Inference: Gun Reform and Lethal Violence - a Difference-in-Difference Analysis
An updated analysis of Cheng and Hoekstra [2013] review of changes to the Castle Doctrine and resulting changes to lethal violence.

# Data Wrangling

## Import Libraries

In [ ]:
# Import pandas, matplotlib.pyplot, and seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load Dataset

In [ ]:
# Accessing Google Drive by mounting it locally
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/GitHub/causal_inference_gun_violence/

In [ ]:
# Location on Google Drive
loc_data = '/content/drive/MyDrive/colab_notebooks/GitHub/causal_inference_gun_violence/'

In [ ]:
# Load Stata datafile into pandas
castle = pd.read_stata(loc_data + 'castle.dta')

In [ ]:
# Unmount Google Drive.
# drive.flush_and_unmount()

## Initial look at dataset

In [ ]:
# Set Pandas options to fully display tables 
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.set_option('max_columns', 9999)

In [ ]:
castle.head()

,state,year,sid,cdl,pre2_cdl,caselaw,anywhere,assumption,civil,homicide_c,robbery_gun_r,jhcitizen_c,jhpolice_c,homicide,robbery,assault,burglary,larceny,motor,murder,hc_felonywsus,jhcitizen,jhpolice,population,police,unemployrt,income,blackm_15_24,whitem_15_24,blackm_25_44,whitem_25_44,prisoner,lagprisoner,poverty,exp_subsidy,exp_pubwelfare,northeast,midwest,south,west,effyear,r20001,r20002,r20003,r20004,r20011,r20012,r20013,r20014,r20021,r20022,r20023,r20024,r20031,r20032,r20033,r20034,r20041,r20042,r20043,r20044,r20051,r20052,r20053,r20054,r20061,r20062,r20063,r20064,r20071,r20072,r20073,r20074,r20081,r20082,r20083,r20084,r20091,r20092,r20093,r20094,r20101,r20102,r20103,r20104,trend_1,trend_2,trend_3,trend_4,trend_5,trend_6,trend_7,trend_8,trend_9,trend_10,trend_11,trend_12,trend_13,trend_14,trend_15,trend_16,trend_17,trend_18,trend_19,trend_20,trend_21,trend_22,trend_23,trend_24,trend_25,trend_26,trend_27,trend_28,trend_29,trend_30,trend_31,trend_32,trend_33,trend_34,trend_35,trend_36,trend_37,trend_38,trend_39,trend_40,trend_41,trend_42,trend_43,trend_44,trend_45,trend_46,trend_47,trend_48,trend_49,trend_50,trend_51,l_murder,l_homicide,l_homicide_c,l_robbery,l_assault,l_burglary,l_larceny,l_motor,l_robbery_gun,l_jhcitizen,l_jhpolice,l_hc_felonywsus,l_robbery_gun_r,l_pop,l_police,l_income,l_prisoner,l_lagprisoner,l_exp_subsidy,l_exp_pubwelfare,treatment_date,post,time_til,lead1,lead2,lead3,lead4,lead5,lead6,lead7,lead8,lead9,lag0,lag1,lag2,lag3,lag4,lag5,_Iyear_2001,_Iyear_2002,_Iyear_2003,_Iyear_2004,_Iyear_2005,_Iyear_2006,_Iyear_2007,_Iyear_2008,_Iyear_2009,_Iyear_2010,popwt
0,Alabama,2000,1,0.0,0.0,0.0,0,0,0,329,0.210803,1,0,7.593978,131.613571,325.617798,930.920166,2940.623779,295.657349,6.532207,0.484722,0.046164,0.023082,4332380,348.838283,4.1,44851,2.222243,4.694810,3.471694,10.414437,605.325500,564.274109,14.705981,212.261169,1018.497009,0,0,1,0,2006.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.876745,2.027356,5.796058,4.879870,5.785724,6.836174,7.986377,5.689201,-1.556830,-3.075555,-3.768702,-0.724180,-1.556830,15.281628,5.854609,10.711102,6.405766,6.335540,5.357818,6.926083,2006.0,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,4499293.0
1,Alabama,2001,1,0.0,0.0,0.0,0,0,0,379,0.214362,2,0,8.713443,128.379593,281.635040,934.384583,2758.689941,290.118561,7.494940,0.666728,0.068972,0.022991,4349601,351.825374,4.7,43301,2.249954,4.700201,3.440247,10.186382,614.792053,605.325500,15.750203,221.890289,1068.928589,0,0,1,0,2006.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.014228,2.164867,5.937536,4.854991,5.640612,6.839888,7.922511,5.670290,-1.540087,-2.674057,-3.772669,-0.405373,-1.540087,15.285595,5.863135,10.675931,6.421284,6.405766,5.402183,6.974412,2006.0,0.0,-5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,4499293.0
2,Alabama,2002,1,0.0,0.0,0.0,0,0,0,303,0.424019,3,2,6.933288,136.423309,274.631409,974.275696,2835.829102,317.832886,6.406999,0.594936,0.091529,0.068646,4370221,311.494545,5.4,45573,2.258261,4.685461,3.403993,9.957734,640.104919,614.792053,15.556559,257.726379,1139.962646,0,0,1,0,2006.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,

In [ ]:
castle.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 550 entries, 0 to 549
Data columns (total 185 columns):
 #    Column            Non-Null Count  Dtype  
---   ------            --------------  -----  
 0    state             550 non-null    object 
 1    year              550 non-null    int16  
 2    sid               550 non-null    int8   
 3    cdl               550 non-null    float32
 4    pre2_cdl          550 non-null    float32
 5    caselaw           550 non-null    float32
 6    anywhere          550 non-null    int8   
 7    assumption        550 non-null    int8   
 8    civil             550 non-null    int8   
 9    homicide_c        550 non-null    int16  
 10   robbery_gun_r     544 non-null    float32
 11   jhcitizen_c       550 non-null    int8   
 12   jhpolice_c        550 non-null    int16  
 13   homicide          550 non-null    float32
 14   robbery           550 non-null    float32
 15   assault           550 non-null    float32
 16   burglary          550 no

In [ ]:
castle.describe().T

,count,mean,std,min,25%,50%,75%,max
year,550.0,2.005000e+03,3.165156e+00,2000.000000,2.002000e+03,2.005000e+03,2.008000e+03,2.010000e+03
sid,550.0,2.634000e+01,1.467979e+01,1.000000,1.400000e+01,2.650000e+01,3.900000e+01,5.100000e+01
cdl,550.0,1.522391e-01,3.476353e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
pre2_cdl,550.0,7.636364e-02,2.658208e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
caselaw,550.0,2.000000e-01,4.003641e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
anywhere,550.0,1.436364e-01,3.510399e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
assumption,550.0,1.109091e-01,3.143054e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
civil,550.0,1.600000e-01,3.669398e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
homicide_c,550.0,3.202600e+02,4.036340e+02,4.000000,4.325000e+01,1.815000e+02,4.307500e+02,2.503000e+03
robbery_gun_r,544.0,3.517144e-01,1.309903e-01,0.008788,2.508814e-01,3.550301e-01,4.600999e-01,5.973725e-01


The 'year' category shows that the dataset goes from the year 2000 to 2010.  

In [ ]:
# Convert year category from int16 to time category.
castle.year.astype('int32')
castle.year = pd.to_datetime(castle.year, format='%Y')

In [ ]:
castle.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 550 entries, 0 to 549
Data columns (total 185 columns):
 #    Column            Non-Null Count  Dtype         
---   ------            --------------  -----         
 0    state             550 non-null    object        
 1    year              550 non-null    datetime64[ns]
 2    sid               550 non-null    int8          
 3    cdl               550 non-null    float32       
 4    pre2_cdl          550 non-null    float32       
 5    caselaw           550 non-null    float32       
 6    anywhere          550 non-null    int8          
 7    assumption        550 non-null    int8          
 8    civil             550 non-null    int8          
 9    homicide_c        550 non-null    int16         
 10   robbery_gun_r     544 non-null    float32       
 11   jhcitizen_c       550 non-null    int8          
 12   jhpolice_c        550 non-null    int16         
 13   homicide          550 non-null    float32       
 14   robbery 

## Characterizing Null Values within the Dataset

In [ ]:
# Dataset Null Value exploration
missing_data = pd.concat([castle.isnull().sum(), 100 * castle.isnull().mean()], axis=1)
missing_data.columns=['count', '%']
missing_data.sort_values(by='count', ascending=False).head(10)

,count,%
time_til,319,58.000000
treatment_date,319,58.000000
effyear,319,58.000000
hc_felonywsus,11,2.000000
l_hc_felonywsus,11,2.000000
l_robbery_gun_r,6,1.090909
l_robbery_gun,6,1.090909
robbery_gun_r,6,1.090909
trend_43,0,0.000000
trend_37,0,0.000000


The Null Values in 'time_til', 'treatment_date', and 'effyear' are from states that did not change their Castle Doctrine laws.

In [ ]:
castle

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
castle.year.min()

Timestamp('2000-01-01 00:00:00')

In [ ]:
castle.year.max()

Timestamp('2010-01-01 00:00:00')

In [ ]:
castle.state.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [ ]:
castle.state.nunique()

50

In [ ]:
castle[['state', 'year', 'time_til', 'treatment_date', 'effyear']]

,state,year,time_til,treatment_date,effyear
0,Alabama,2000-01-01,-6.0,2006.0,2006.0
1,Alabama,2001-01-01,-5.0,2006.0,2006.0
2,Alabama,2002-01-01,-4.0,2006.0,2006.0
3,Alabama,2003-01-01,-3.0,2006.0,2006.0
4,Alabama,2004-01-01,-2.0,2006.0,2006.0
5,Alabama,2005-01-01,-1.0,2006.0,2006.0
6,Alabama,2006-01-01,0.0,2006.0,2006.0
7,Alabama,2007-01-01,1.0,2006.0,2006.0
8,Alabama,2008-01-01,2.0,2006.0,2006.0
9,Alabama,2009-01-01,3.0,2006.0,2006.0


## Data Wrangling Conclusions

From 2000 to 2010, 21 states adopted laws that changed Castle Doctrine from Duty to Retreat to Stand Your Ground.  This change in the law is the treatment, in an economic definition of the term.  The treatment_date and effyear variables record when the treatment occurred.  

This dataset is derived from the Uniform Crime Reporting Program (https://www.fbi.gov/how-we-can-help-you/more-fbi-services-and-information/ucr) that tracks 8 criminal stats across all 50 states since the 1920s - murder and non-negligent manslaughter, forcible rape, burglary, aggravated assault, larceny, motor vehicle theft, and arson.  These statistics are converted into rates - incidents per 100,000 population.  

These variables are further converted to log rates (l_homicide for example) for use in regression.  North/South/East/West are dummy variables for census regions.  blackm_25_44, etc. are statistics to characterize population.  r20001, etc. are dummy variables for regressions.  

Null values are related to states that never changed there gun laws and are acting as a control.  Therefore, these are valid null values and do not need to be imputed.